In [1]:
import networkx as nx
from networkx.generators.community import LFR_benchmark_graph

In [4]:
n = 100000
tau1 = 3 # 2-3
tau2 = 1.5 # 1-2
mu = 0.1

def create_benchmark_graph(n, tau1, tau2, mu, average_degree, min_community, seed):
    G = LFR_benchmark_graph(
        n, tau1, tau2, mu, average_degree=average_degree, \
            min_community=min_community, seed=seed
    )
    return G


In [42]:
configuration = {
    'n':[ 10000],
    'tau1': [2],
    'tau2': [1.5],
    'mu': [0.1],
    'avg_degree': [ 16]
}

In [43]:

configurations_possible = []

def get_possible_configs(conf, index, current):

    if index == len(conf):
        configurations_possible.append(current)
        return 

    here = list(conf.keys())[index]

    for value in conf[here]:
        current[here] = value
        get_possible_configs(conf, index + 1, current)


get_possible_configs(configuration, 0, {})


In [82]:
from collections import defaultdict
import community.community_louvain as community_louvain

def get_communities(G):

    communities = {frozenset(G.nodes[v]["community"]) for v in G}
    # (com_nr, set community)
    lrf_dict = defaultdict()
    c = 1
    for y in communities:
        lrf_dict[c] = y
        c+= 1


    # (comm_nr, set communities)
    partion = community_louvain.best_partition(G)
    louvain_dict = defaultdict(set)
    for node, community in partion.items():
        louvain_dict[community].add(node) # (community_nr, nodes)

    return lrf_dict, louvain_dict



def get_percentage_intercetion(set_ids, mapping_ids):

    z = set()
    for id in set_ids:
        z = z.union(mapping_ids[id])
    return z






def compare_communities_stats(G):

    lrf_dict, louvain_dict = get_communities(G)
    assigned_ones = defaultdict(set)# sets from the louvain that are assigned as most similar to the same set of the lrf

    for k, y in lrf_dict.items():
        
        higher = -1
        target_set = set() # set with the most common nodes
        overlapping_set_nr = set()

        for nr, com_set in louvain_dict.items():

            temp = com_set & y
            
            if len(temp) > 0:
                overlapping_set_nr.add(nr)
                assigned_ones[nr].add(k)

            
            if higher < len(temp):
                higher = len(temp)

        louvain_union = get_percentage_intercetion(overlapping_set_nr, louvain_dict)
        lrf_union = lrf_dict[k]
        percentage = len(louvain_union.intersection(lrf_union)) / max(len(louvain_union), len(lrf_union))
        #print("For lrf community with size {}, there were found in louvain {} communities overlapping".format(len(y), len(overlapping_set_nr)))
        print("For lrf community id {}, they intersect with louvain communities ids {} => {}".format(k, overlapping_set_nr, percentage))


    print("the other way around:")
    for k, y in assigned_ones.items(): 
        # k -> community id of louvain
        # y -> set of communities id of lrf
        # community id of louvain -> communties of lrf that overlap with the louvain
        louvain_union = louvain_dict[k]
        lrf_union = get_percentage_intercetion(y, lrf_dict)
        percentage = len(louvain_union.intersection(lrf_union)) / max(len(louvain_union), len(lrf_union))
        print("{} (louvain) : {} (lrf) => {}" .format(k, y, percentage))

def compare_communities_number(G):
    partion = community_louvain.best_partition(G)
    l_c = max(partion.values())

    communities = {frozenset(G.nodes[v]["community"]) for v in G}
    lrf_c = len(communities)

    return l_c , lrf_c



In [83]:
def run_experiment(configurations_possible, experiment_call_back):

    values = []
    for configuration in configurations_possible:
        print(configuration)
        G = create_benchmark_graph(configuration['n'], configuration['tau1'], \
            configuration['tau2'], configuration['mu'], \
            average_degree=configuration['avg_degree'], min_community=20, seed=10)
        values.append(experiment_call_back(G))
    
    print(values)

run_experiment(configurations_possible, compare_communities_stats)

{'n': 10000, 'tau1': 2, 'tau2': 1.5, 'mu': 0.1, 'avg_degree': 16}
For lrf community id 1, they intersect with louvain communities ids {1} => 0.9985666507405638
For lrf community id 2, they intersect with louvain communities ids {10} => 0.9555555555555556
For lrf community id 3, they intersect with louvain communities ids {14} => 0.9809523809523809
For lrf community id 4, they intersect with louvain communities ids {4} => 0.6428571428571429
For lrf community id 5, they intersect with louvain communities ids {13} => 0.9836065573770492
For lrf community id 6, they intersect with louvain communities ids {9} => 1.0
For lrf community id 7, they intersect with louvain communities ids {0, 1, 2, 4, 5, 6, 7, 8, 10, 13, 14, 16} => 0.581657113079729
For lrf community id 8, they intersect with louvain communities ids {16} => 0.9846153846153847
For lrf community id 9, they intersect with louvain communities ids {4, 5} => 0.0035790980672870437
For lrf community id 10, they intersect with louvain comm

# numero de comunidades parece manter-se constante